## Apiux & SII: Analisis exploratorio de datos y depuracion de data sociedades
## ATENCION: proyecto sujeto a mantenimiento continuo. 

## Henry Vega (henrry.vega@api-ux.com)
## Data analyst

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
import pyspark
import pandas as pd
#warnings.filterwarnings('ignore', category=DeprecationWarning)

In [2]:

spark = SparkSession.builder \
  .appName("Test")  \
  .config("spark.yarn.access.hadoopFileSystems","abfs://data@datalakesii.dfs.core.windows.net/") \
  .config("spark.executor.memory", "24g") \
  .config("spark.driver.memory", "12g")\
  .config("spark.executor.cores", "12") \
  .config("spark.executor.instances", "24") \
  .config("spark.driver.maxResultSize", "12g") \
  .getOrCreate()

Setting spark.hadoop.yarn.resourcemanager.principal to yasser.nanjari


## Carga de relaciones societarias y depuracion de data

Primero, veamos los valores null en participacion de capital y participacion de utilidades.

In [3]:
spark.sql("select RUT_SOCIEDAD, RUT_SOCIO,PORCENTAJE_CAPITAL,PORCENTAJE_UTILIDADES from libsdf.jab_soc_2023_inom where PORCENTAJE_CAPITAL is null or PORCENTAJE_UTILIDADES IS NULL").show()

Hive Session ID = 1ac9261b-b513-43b9-9809-2754382080fd


+------------+---------+------------------+---------------------+
|RUT_SOCIEDAD|RUT_SOCIO|PORCENTAJE_CAPITAL|PORCENTAJE_UTILIDADES|
+------------+---------+------------------+---------------------+
+------------+---------+------------------+---------------------+



Vemos que no hay valores nulos en la tabla. A continuacion veamos cuantos duplicados existen en las columnas de interes:
RUT_SOCIEDAD,RUT_SOCIO,PORCENTAJE_CAPITAL,PORCENTAJE_UTILIDADES, pues todos
los calculos lo haremos basados en esta columnas.

In [4]:
spark.sql("select * from libsdf.jba_soc_2022_e where RUT_SOCIEDAD like 'dwElNqcCQQiFyI3ic%' ").show()

[Stage 5:==================================================>        (6 + 1) / 7]

+------------+---------+------------------+---------------------+------------------------+------+
|RUT_SOCIEDAD|RUT_SOCIO|PORCENTAJE_CAPITAL|PORCENTAJE_UTILIDADES|PERI_AGNO_MES_TRIBUTARIO|FUENTE|
+------------+---------+------------------+---------------------+------------------------+------+
+------------+---------+------------------+---------------------+------------------------+------+



In [5]:
spark.sql("select RUT_SOCIEDAD, RUT_SOCIO,PORCENTAJE_CAPITAL,PORCENTAJE_UTILIDADES, count(*) as c from libsdf.jba_soc_2023_e  group by  RUT_SOCIEDAD, RUT_SOCIO,PORCENTAJE_CAPITAL,PORCENTAJE_UTILIDADES order by c desc").createOrReplaceTempView("sociedad")
spark.sql("select * from sociedad").show()
spark.sql("select RUT_SOCIEDAD, RUT_SOCIO,PORCENTAJE_CAPITAL,PORCENTAJE_UTILIDADES from sociedad").createOrReplaceTempView("sociedad")

[Stage 7:=========================>                             (93 + 12) / 200]

+--------------------+--------------------+------------------+---------------------+---+
|        RUT_SOCIEDAD|           RUT_SOCIO|PORCENTAJE_CAPITAL|PORCENTAJE_UTILIDADES|  c|
+--------------------+--------------------+------------------+---------------------+---+
|lvhKR91n05T3mstWg...|tnshXz2zZHhF/35iM...|             100.0|                100.0| 33|
|FJ59A3rCUrWEuEI/g...|AtxwrhZVZnI1sYNt/...|              50.0|                 50.0| 15|
|U9dRqzVEukCZdQmX8...|qlm4skAGTxB/gAmri...|             100.0|                100.0| 14|
|gwbsyFNCMsaytOcg8...|NvQrSPRmMfK9N1riN...|               0.0|                  0.0| 13|
|51GDT15LxNxXvHK+5...|rhswzqGrtBdFr5AJW...|              60.0|                 60.0| 12|
|jivM6l8xeUjKSU5fI...|26fBX92ujXPn2atcj...|             100.0|                100.0| 12|
|YERESpwuZelDrFKfk...|aefg+HqgBVthzV/Dp...|              20.0|                 20.0| 12|
|/ZF4A1Ysc/R8tPQnl...|RVjWLn8FlBViX6I4N...|             100.0|                100.0| 12|
|g/CiBABMUw9aF9wzh...

Donde seleccionamos los valores no repetidos. Haciendo nuevamente un recuento de los valores unicos,

In [6]:
spark.sql("select RUT_SOCIEDAD, RUT_SOCIO, count(*) as count from sociedad group by RUT_SOCIEDAD, RUT_SOCIO order by count desc ").show()

[Stage 12:===========================================>              (3 + 1) / 4]

+--------------------+--------------------+-----+
|        RUT_SOCIEDAD|           RUT_SOCIO|count|
+--------------------+--------------------+-----+
|cfh2XO8QryKKtyO4M...|qJhovaa1JWqL7YY/f...|    1|
|gwbsyFNCMsaytOcg8...|HSkI6zeV4EXsc1DHI...|    1|
|cyysdP+Ve7dCq9WSa...|mpxPjETO9I0AN7PH/...|    1|
|G9IHbpdnVGiyoja+v...|won8MWCfBnNVA+H7J...|    1|
|aOMC0QM64KeOvLehs...|kT8TTtPbVneTpnnUg...|    1|
|7PzGWLJc/467GGYaC...|xGYI+bX6pAINJTEGc...|    1|
|y88ag0hm9sdwdYSCn...|rBernrFpDxswe9dVq...|    1|
|SC/oNmnMA/oR2Lamj...|vOo57GpxCw2WgiZoD...|    1|
|f58bGiUzOCx8zCvt2...|oZxHsGLH04wQjNIlw...|    1|
|In/OT2MgGS2uvTc03...|7YqVygUMhoVOiyCS4...|    1|
|j9uffvDYsLMxS/7IQ...|KxN6MptI2pZGsVRu6...|    1|
|oRDDo8yg6eCfHXJEz...|EgkN9xUgEqgwNJw70...|    1|
|V9AQpN7N+fcocd59i...|bvLSOzVrHGe2Ts8hL...|    1|
|byrZOz15Ks2dy5xv4...|/YF1gxxzh2Z1s4tsB...|    1|
|t3KX0aVeH4tjc6RL7...|43jDeKcoOn/TvMb6w...|    1|
|h6T6lD/Pxw+dM+MO5...|ohB5PPr6HI0kxCBsH...|    1|
|MHTWlmWP5c/31n2rj...|PGnTFmv5v5n/C0+lL...|    1|


Por lo visto tenemos unicidad de la relaciones sociedad socio, a juzgar por el recuento de las combinaciones. Ahora veremos cuanto suman los valores de PORCENTAJE_CAPITAL,PORCENTAJE_UTILIDADES para cada una de las sociedades.


In [7]:
spark.sql("select RUT_SOCIEDAD, SUM(PORCENTAJE_CAPITAL) as CAPITAL,SUM(PORCENTAJE_UTILIDADES) as UTILIDADES from sociedad group by RUT_SOCIEDAD order by CAPITAL DESC").show()

[Stage 18:===========================================>              (3 + 1) / 4]

+--------------------+-------+----------+
|        RUT_SOCIEDAD|CAPITAL|UTILIDADES|
+--------------------+-------+----------+
|FtJmDwwsdBDC15rvW...|  110.0|     110.0|
|HWeUcxFfs+xB+tv4B...|  110.0|     100.0|
|f1zMRTsAknUF9weV0...|  110.0|     110.0|
|+P/D3dLQdBXBfBBMk...|  110.0|     110.0|
|tMloWBszd+pnK6pjz...|  110.0|     110.0|
|yBR0RNSMY5VYroBj3...|  110.0|     110.0|
|n5MD+Ght8Cx8X+xhK...|  110.0|     110.0|
|frSs4/GZOUTCL9+U+...|  110.0|     110.0|
|KmKCw9Hb8gOrM1Z9W...|  110.0|     100.0|
|VRNdJkUTYXLWMRkSM...|  110.0|     110.0|
|HmjHOVQ8qk9aGu30J...|  110.0|     110.0|
|CP3xFFso/QqV9VaqJ...|  110.0|     110.0|
|EpHOFngzeSBjKqble...|  110.0|     110.0|
|s+CDuiI+aTB8rRFTE...|  110.0|     110.0|
|MSvsNkqEo2nh7yGTH...|  110.0|     110.0|
|EqLVYjdaxkTliFuxD...|  110.0|     110.0|
|4ycAGBELEjW5Sts2g...|  110.0|     100.0|
|VQTk89mGDDDvTgvgP...|  110.0|     110.0|
|ZWuslkhcOMdiyrD/W...|  110.0|     110.0|
|PQS6x5nDlb1V3Kt8s...|  110.0|     100.0|
+--------------------+-------+----

In [8]:
spark.sql("select RUT_SOCIEDAD, SUM(PORCENTAJE_CAPITAL) as CAPITAL,SUM(PORCENTAJE_UTILIDADES) as UTILIDADES from sociedad group by RUT_SOCIEDAD order by CAPITAL ASC").show()

[Stage 24:===========================================>              (3 + 1) / 4]

+--------------------+------------------+------------------+
|        RUT_SOCIEDAD|           CAPITAL|        UTILIDADES|
+--------------------+------------------+------------------+
|gwbsyFNCMsaytOcg8...|11.289999999999841|11.289999999999841|
|2laNNn7TvcvjsHi5t...|              90.0|              90.0|
|hNmKzKIHSTIMo1i0W...|              90.0|              90.0|
|XaSTQH0ulw3G7F2yz...|              90.0|              90.0|
|1YVpvhCjcGUZLc9mk...|              90.0|              90.0|
|h3pCmA3K4viqJHTYM...|              90.0|              90.0|
|eWKG01GX1QZnp6rsL...|              90.0|              90.0|
|vsQK6z66k54TaKb/H...|              90.0|              90.0|
|aZjOa91WeJhn7EUjg...|              90.0|              90.0|
|jcpaORx9K8u8sw7x0...|              90.0|              90.0|
|yQNTXOOh78ZNrUMLu...|              90.0|              90.0|
|QHlfqsVnl5ihsUAWl...|              90.0|              90.0|
|pXvrYgUFd1fJaX9eK...|              90.0|              90.0|
|TWksGxJMctALmuUdh...|  

In [9]:
anomalias_sociedades=spark.sql("select RUT_SOCIEDAD, SUM(PORCENTAJE_CAPITAL) as CAPITAL,SUM(PORCENTAJE_UTILIDADES) as UTILIDADES from sociedad group by RUT_SOCIEDAD having CAPITAL>105 or CAPITAL<95").toPandas()
anomalias_sociedades.to_csv('../../data/processed/anomalias_capital_sociedades.csv', index=False)
print(anomalias_sociedades.describe())

[Stage 30:===========================================>              (3 + 1) / 4]

          CAPITAL  UTILIDADES
count  648.000000  648.000000
mean    93.608534   93.416065
std      7.510806    7.918392
min     11.290000   11.290000
25%     90.000000   90.000000
50%     90.000000   90.000000
75%     93.207500   93.917500
max    110.000000  127.000000


Ahora vamos las entradas con al menos un valor cero (que indica cero participacion porcentual)

In [10]:
spark.sql("select COUNT(*) from sociedad WHERE PORCENTAJE_CAPITAL=0 OR PORCENTAJE_UTILIDADES=0").show()

+--------+
|count(1)|
+--------+
| 1060913|
+--------+



Por lo visto, tenemos 1057765 entradas donde al menos uno de ambos porcentajes es cero. Por otro lado, tenemos 
1053936 registros donde ambos porcentajes son cero.


Ahora veamos cuales porcentajes de capital son cero y luego los porcentajes de utilidades son cero.

In [11]:
spark.sql("select count(*) from sociedad WHERE PORCENTAJE_CAPITAL=0 and PORCENTAJE_UTILIDADES!=0").show()
spark.sql("select count(*) from sociedad WHERE PORCENTAJE_CAPITAL!=0 and PORCENTAJE_UTILIDADES=0").show()

+--------+
|count(1)|
+--------+
|    2484|
+--------+



+--------+
|count(1)|
+--------+
|    1231|
+--------+



Para el analisis del problema de oscuridad, es mejor tener en cuenta los porcentajes de participacion de capital, porque los creditos se reparten segun la participacion societaria.
Ahora veamos cuantos tienen valores positivos mayores que 100 o negativos.

In [12]:
spark.sql("select * from sociedad WHERE PORCENTAJE_CAPITAL<0 or PORCENTAJE_CAPITAL>100 or PORCENTAJE_UTILIDADES<0 or PORCENTAJE_UTILIDADES>100").show()

[Stage 50:======================================================> (32 + 1) / 33]

+--------------------+--------------------+------------------+---------------------+
|        RUT_SOCIEDAD|           RUT_SOCIO|PORCENTAJE_CAPITAL|PORCENTAJE_UTILIDADES|
+--------------------+--------------------+------------------+---------------------+
|7ZfJs1v+u1B0SCmQJ...|K4PMvCs8vXhziffJN...|            100.01|                100.0|
+--------------------+--------------------+------------------+---------------------+



El cual es solo un valor levemente superior a 100 %. Seleccionamos los que no tienen valores cero en PORCENTAJE_CAPITAL.
IMPORTANTE: de ser utilizado PORCENTAJE_UTILIDADES se debe filtrar sobre esa columna.

In [13]:
spark.sql("select * from sociedad where PORCENTAJE_CAPITAL!=0").createOrReplaceTempView("sociedad")

In [14]:
spark.sql("select count(*) from sociedad where RUT_SOCIEDAD LIKE 'Qbau/6SlJ/lEcKUD%'").show()
spark.sql("select *  from libsdf.jba_soc_2022_e where RUT_SOCIEDAD LIKE 'Qbau/6SlJ/lEcKUD%'").show()

+--------+
|count(1)|
+--------+
|       0|
+--------+



[Stage 60:=====================================================>  (19 + 1) / 20]

+------------+---------+------------------+---------------------+------------------------+------+
|RUT_SOCIEDAD|RUT_SOCIO|PORCENTAJE_CAPITAL|PORCENTAJE_UTILIDADES|PERI_AGNO_MES_TRIBUTARIO|FUENTE|
+------------+---------+------------------+---------------------+------------------------+------+
+------------+---------+------------------+---------------------+------------------------+------+



In [15]:

sociedad=spark.sql("select * from sociedad").toPandas()
sociedad.to_csv('../../data/processed/sociedades_participacion_capital_nozero.csv', index=False)